## test prediction from saved model

In [ ]:
from skimage import io
import numpy as np

model_path = r"C:\Users\ASUS\Jupyter Notebook\Fruit and Vegetable Freshness Classification\fruit-vegetable-freshness\IDGAF\models\fruit-veg-freshness-classification\fruit_veg_freshness.h5"
img_path = r"C:\Users\ASUS\Downloads\download.jpg"

saved_model = tf.keras.models.load_model(model_path)

img = io.imread(img_path)
img = np.resize(pic, (224,224,3))
img = np.expand_dims(img, axis=0)

saved_model.predict(img)

## deploy to local server

In [2]:
import base64
import json
from io import BytesIO
import numpy as np
import requests
from flask import Flask, request, jsonify
import tensorflow as tf

# load model
model_path = r"C:\Users\ASUS\Jupyter Notebook\Fruit and Vegetable Freshness Classification\fruit-vegetable-freshness\IDGAF\models\fruit-veg-freshness-classification\fruit_veg_freshness.h5"
model = tf.keras.models.load_model(model_path)

# input image specification
img_height = 224
img_width = 224
img_size = (img_height, img_width)
img_shape = img_size + (3,)

# classes to predict
class_names = ['f_apple', 'f_banana', 'f_carrot', 'f_greens', 'f_orange', 'f_tomato',
'm_apple', 'm_banana', 'm_carrot', 'm_greens', 'm_orange', 'm_tomato', 'r_apple',
'r_banana', 'r_carrot', 'r_greens', 'r_orange', 'r_tomato']


# simple flask web app
app = Flask(__name__)
@app.route("/")
def test():
    return "OK"

# prediction
@app.route("/predict", methods=["POST"])
def predict():
    # load image & preprocess
    pic = np.array(tf.keras.utils.load_img(BytesIO(base64.b64decode(request.form['b64'])),
                                        target_size=img_shape))
    pic = np.expand_dims(pic, axis=0)
    pic = pic.astype('float16')
    
    # predict image class
    pred = model.predict(pic)
    pred_idx = np.argmax(pred, axis=-1)
    prediction = "it's a {}!".format(class_names[int(pred_idx)])
    response_json = {"prediction" : prediction}
    return json.dumps(response_json)

# run app
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


1/1 [==============================] - 4s 4s/step


127.0.0.1 - - [01/Jun/2022 11:45:03] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 245ms/step


127.0.0.1 - - [01/Jun/2022 11:47:00] "POST /predict HTTP/1.1" 200 -


## request prediction
run in another file

In [ ]:
import requests
import json
import base64
from skimage import io
import matplotlib.pyplot as plt

img_path = r"C:\Users\ASUS\Downloads\download.jpg"
img = io.imread(img_path)

with open(img_path, 'rb') as file:
    test_img = base64.b64encode(file.read())
    
data = {'b64': test_img}
endpoint = "http://127.0.0.1:5000/predict"
resp = requests.post(endpoint, data=data)

print(resp.json())
plt.imshow(img);